In [154]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, glob
import numpy as np
from tqdm import tqdm
import torch
import smplx
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.transform import Rotation as R
from scipy.ndimage import gaussian_filter1d
import json
import csv
import pdb


In [155]:
def get_body_model(type, gender, batch_size,device='cpu'):
    '''
    type: smpl, smplx smplh and others. Refer to smplx tutorial
    gender: male, female, neutral
    batch_size: an positive integar
    '''
    body_model_path = '/home/yuxinyao/body_models/'
    body_model = smplx.create(body_model_path, model_type=type,
                                    gender=gender, ext='npz',
                                    num_pca_comps=12,
                                    create_global_orient=True,
                                    create_body_pose=True,
                                    create_betas=True,
                                    create_left_hand_pose=True,
                                    create_right_hand_pose=True,
                                    create_expression=True,
                                    create_jaw_pose=True,
                                    create_leye_pose=True,
                                    create_reye_pose=True,
                                    create_transl=True,
                                    batch_size=batch_size
                                    )
    if device == 'cuda':
        return body_model.cuda()
    else:
        return body_model

In [156]:
#### set input output dataset paths
amass_dataset_path = '/home/yuxinyao/datasets/AMASS/data'
egobody_dataset_path = '/home/yuxinyao/datasets/egobody/smplx_interactee'
egobody_smplx_path = '/home/yuxinyao/datasets/egobody/egobody-Canonicalized-locomotion-MP/smplx_interactee'
# amass_subsets = ['egobody']


## set mosh markers
## read the corresponding smplx verts indices as markers.
with open('/home/yuxinyao/body_models/Mosh_related/CMU.json') as f:
        marker_cmu_41 = list(json.load(f)['markersets'][0]['indices'].values())

with open('/home/yuxinyao/body_models/Mosh_related/SSM2.json') as f:
        marker_ssm_67 = list(json.load(f)['markersets'][0]['indices'].values())

bm_one_male = get_body_model('smplx','male',1)
bm_one_female = get_body_model('smplx','female',1)

In [221]:
# subset = 'egobody'
folder_seqs = glob.glob(os.path.join(egobody_dataset_path,"recording*"))
seqs = glob.glob(os.path.join(folder_seqs[0], '*/*/*/*.pkl'))

outfolder = os.path.join(egobody_dataset_path)


In [223]:
# seqs

In [158]:
def calc_calibrate_offset(body_mesh_model, betas, transl, pose):
    '''
    The factors to influence this offset is not clear. Maybe it is shape and pose dependent.
    Therefore, we calculate such delta_T for each individual body mesh.
    It takes a batch of body parameters
    input:
        body_params: dict, basically the input to the smplx model
        smplx_model: the model to generate smplx mesh, given body_params
    Output:
        the offset for params transform
    '''
    n_batches = transl.shape[0]
    bodyconfig = {}
    bodyconfig['body_pose'] = torch.FloatTensor(pose).cuda()
    bodyconfig['betas'] = torch.FloatTensor(betas).repeat(n_batches,1).cuda()
    bodyconfig['transl'] = torch.zeros([n_batches,3], dtype=torch.float32).cuda()
    bodyconfig['global_orient'] = torch.zeros([n_batches,3], dtype=torch.float32).cuda()
    
    smplx_out = body_mesh_model(return_verts=True, **bodyconfig)
    delta_T = smplx_out.joints[:,0,:] # we output all pelvis locations
    delta_T = delta_T.detach().cpu().numpy() #[t, 3]

    return delta_T




def get_new_coordinate(body_mesh_model, betas, transl, pose, global_orient):
    '''
    this function produces transform from body local coordinate to the world coordinate.
    it takes only a single frame.
    local coodinate:
        - located at the pelvis
        - x axis: from left hip to the right hip
        - z axis: point up (negative gravity direction)
        - y axis: pointing forward, following right-hand rule
    '''
    bodyconfig = {}
    bodyconfig['transl'] = torch.FloatTensor(transl)
    bodyconfig['global_orient'] = torch.FloatTensor(global_orient)
    bodyconfig['body_pose'] = torch.FloatTensor(pose)
    bodyconfig['betas'] = torch.FloatTensor(betas)
    print("transl:{}, global_orient:{}, body_pose;{},betas:{}".format( bodyconfig['transl'].shape, bodyconfig['global_orient'].shape, bodyconfig['body_pose'].shape, bodyconfig['betas'].shape))
    smplxout = body_mesh_model(**bodyconfig)
    joints = smplxout.joints.squeeze().detach().cpu().numpy()
    # faces = body_mesh_model.faces.detach().cpu().numpy()
    # v = body_mesh_model.vertices.detach().cpu().numpy()

    x_axis = joints[2,:] - joints[1,:]
    x_axis[-1] = 0
    x_axis = x_axis / np.linalg.norm(x_axis)
    z_axis = np.array([0,0,1])
    y_axis = np.cross(z_axis, x_axis)
    y_axis = y_axis/np.linalg.norm(y_axis)
    global_ori_new = np.stack([x_axis, y_axis, z_axis], axis=1)
    transl_new = joints[:1,:] # put the local origin to pelvis

    return global_ori_new, transl_new




def get_body_model(type, gender, batch_size,device='cpu'):
    '''
    type: smpl, smplx smplh and others. Refer to smplx tutorial
    gender: male, female, neutral
    batch_size: an positive integar
    '''
    body_model_path = '/home/yuxinyao/body_models/'
    body_model = smplx.create(body_model_path, model_type=type,
                                    gender=gender, ext='npz',
                                    num_pca_comps=12,
                                    create_global_orient=True,
                                    create_body_pose=True,
                                    create_betas=True,
                                    create_left_hand_pose=True,
                                    create_right_hand_pose=True,
                                    create_expression=True,
                                    create_jaw_pose=True,
                                    create_leye_pose=True,
                                    create_reye_pose=True,
                                    create_transl=True,
                                    batch_size=batch_size
                                    )
    if device == 'cuda':
        return body_model.cuda()
    else:
        return body_model


import sys

In [246]:
data.keys()

dict_keys(['betas', 'global_orient', 'transl', 'left_hand_pose', 'right_hand_pose', 'jaw_pose', 'leye_pose', 'reye_pose', 'expression', 'pose_embedding', 'body_pose', 'gender'])

In [546]:
import pickle

index_subseq = 0 # index subsequences for subsets separately
#### main loop to process each sequence
bm_batch_male = 0
bm_batch_female = 0
bodymodel_batch = 0
bodymodel_one = 0
# params = np.load('/home/yuxinyao/datasets/egobody/test/recording_20210910_S06_S05_01/subseq_00000.npz')
# ## define body model according to gender


# for seq in tqdm(seqs):
#     ## read data
#     # if os.path.basename(seq) == 'shape.npz':
#     #     continue
len_subseq = 10
bm_one_male = get_body_model('smplx','male',1)
bm_one_female = get_body_model('smplx','female',1)
bm_batch_male = get_body_model('smplx','male',len_subseq,device='cuda')
bm_batch_female = get_body_model('smplx','female',len_subseq,device='cuda')

# for record in folder_seqs[:5]:
#     print(folder_seqs[0])
#     print(record)
#     seqs = glob.glob(os.path.join(record, '/*/*/*/*.npz'))
#     print(seqs[0])
for record in folder_seqs:
    seqs = glob.glob(os.path.join(record, '*/*/*/*.pkl'))
    # print(record)
    # print(seqs)
    for i in range(0, len(seqs),len_subseq):
        first_frame_path = seqs[i]
        with open(first_frame_path,'rb') as f:
            first_frame = dict(pickle.load(f))
        transl_all = np.zeros((len_subseq,3))
        pose_all = np.zeros((len_subseq,63))
        betas = first_frame['betas']
        global_orient_all = np.zeros((len_subseq,3))
        # print(betas)
        bodymodel_batch = bm_batch_male if str(first_frame['gender']) =='male' else bm_batch_female
        bodymodel_one = bm_one_male if str(first_frame['gender']) =='male' else bm_one_female
        # print("i:"+str(i))
        for j in range(0,len_subseq):
            seq = seqs[j]
            with open(seq,'rb') as f:
                data = dict(pickle.load(f))
            ## read data
            transl = data['transl']
            pose = data['body_pose']
            global_orient = data['global_orient']
            transl_all[j,:] = transl
            pose_all[j,:]= pose
            global_orient_all[j,:] = global_orient

        
        data_out = {}

        ## break if remaining frames are not sufficient
        # if transl.shape[0] < len_subseq:
        #     break
        # outfolder = os.path.join("/home/yuxinyao/datasets/egobody/test/",record)
        outfolder = "/home/yuxinyao/datasets/egobody/test/" + str(record[49:])
        if not os.path.exists(outfolder):
            os.makedirs(outfolder)
        # print('-- processing subset {:s}'.format("egobody"))
        print(outfolder)
        outfilename = os.path.join(outfolder, 'subseq_{:05d}.npz'.format(i))
        print("!!!!!"+outfilename)
        ## perform transformation from the world coordinate to the amass coordinate
        ### get transformation from amass space to world space
        # print("betas[:10].shape:{}, transl[:1,:].shape:{}, pose[:1]:{},global_orient[:1,:]:{}".format(betas[:10].shape, transl[:1,:].shape, pose[:1].shape, global_orient[:1,:].shape))
        transf_rotmat, transf_transl = get_new_coordinate(bodymodel_one, betas[:10], transl[:1,:], pose[:1],global_orient[:1,:])
        ### calibrate offset
        delta_T = calc_calibrate_offset(bodymodel_batch, betas[:10], transl_all, pose_all)
        ### get new global_orient
        global_ori = R.from_rotvec(global_orient_all).as_matrix() # to [t,3,3] rotation mat
        global_ori_new = np.einsum('ij,tjk->tik', transf_rotmat.T, global_ori)
        global_ori_save = R.from_matrix(global_ori_new).as_rotvec()
        ### get new transl
        transl = np.einsum('ij,tj->ti', transf_rotmat.T, transl_all+delta_T-transf_transl)-delta_T
        data_out['transf_rotmat'] = transf_rotmat
        data_out['transf_transl'] = transf_transl
        data_out['trans'] = transl
        data_out['poses'] = pose_all
        data_out['betas'] = betas
        data_out['gender'] = data['gender']
        data_out['global_orient'] = global_ori_save
        data_out['mocap_framerate'] = 30

        ## under this new amass coordinate, extract the joints/markers' locations
        ## when get generated joints/markers, one can directly transform them back to world coord
        ## note that hand pose is not considered here. In amass, the hand pose is regularized.
        body_param = {}
        body_param['transl'] = torch.FloatTensor(transl).cuda()
        body_param['global_orient'] = torch.FloatTensor(global_orient_all).cuda()
        body_param['betas'] = torch.FloatTensor(betas[:10]).repeat(len_subseq,1).cuda()
        body_param['body_pose'] = torch.FloatTensor(pose_all).cuda()
        smplxout = bodymodel_batch(return_verts=True, **body_param)
        
        ### extract joints and markers
        joints = smplxout.joints[:,:22,:].detach().squeeze().cpu().numpy()
        # faces = smplxout.faces.detach().squeeze().cpu().numpy()
        # vertices = smplxout.vertices.detach().squeeze().cpu().numpy()
        markers_41 = smplxout.vertices[:,marker_cmu_41,:].detach().squeeze().cpu().numpy()
        markers_67 = smplxout.vertices[:,marker_ssm_67,:].detach().squeeze().cpu().numpy()
        data_out['joints'] = joints
        data_out['marker_cmu_41'] = markers_41
        data_out['marker_ssm2_67'] = markers_67
        # data_out['faces'] = faces
        # data_out['vertices'] = vertices

        np.savez(outfilename, **data_out)
        # t = t+len_subseq
        # index_subseq = index_subseq+1








/home/yuxinyao/datasets/egobody/test/recording_20210910_S06_S05_01
!!!!!/home/yuxinyao/datasets/egobody/test/recording_20210910_S06_S05_01/subseq_00000.npz
transl:torch.Size([1, 3]), global_orient:torch.Size([1, 3]), body_pose;torch.Size([1, 63]),betas:torch.Size([1, 10])

        global_orient.reshape(-1, 1, 3): [torch.Size([1, 1, 3]))],
        body_pose.reshape(-1, self.NUM_BODY_JOINTS, 3):[torch.Size([1, 21, 3])],
        jaw_pose.reshape(-1, 1, 3): [torch.Size([1, 1, 3])]
        leye_pose.reshape(-1, 1, 3): [torch.Size([1, 1, 3])],
        reye_pose.reshape(-1, 1, 3),
        left_hand_pose.reshape(-1, 15, 3),
        right_hand_pose.reshape(-1, 15, 3)]

        global_orient.reshape(-1, 1, 3): [torch.Size([10, 1, 3]))],
        body_pose.reshape(-1, self.NUM_BODY_JOINTS, 3):[torch.Size([10, 21, 3])],
        jaw_pose.reshape(-1, 1, 3): [torch.Size([10, 1, 3])]
        leye_pose.reshape(-1, 1, 3): [torch.Size([10, 1, 3])],
        reye_pose.reshape(-1, 1, 3),
        left_hand_po

AttributeError: 'int' object has no attribute 'vertices'

In [241]:
len("/home/yuxinyao/datasets/egobody/smplx_interactee/")

49

In [240]:
record

'/home/yuxinyao/datasets/egobody/smplx_interactee/recording_20220225_S24_S25_02'

In [244]:
outfolder = "/home/yuxinyao/datasets/egobody/test/" + str(record[49:])

In [267]:
data.keys()

dict_keys(['betas', 'global_orient', 'transl', 'left_hand_pose', 'right_hand_pose', 'jaw_pose', 'leye_pose', 'reye_pose', 'expression', 'pose_embedding', 'body_pose', 'gender'])

In [256]:
params.keys()

dict_keys(['transf_rotmat', 'transf_transl', 'trans', 'poses', 'betas', 'gender', 'mocap_framerate', 'joints', 'marker_cmu_41', 'marker_ssm2_67'])

In [ ]:
# cmu = dict(np.load()

In [261]:
transl = params['trans']
pose = params['poses']
beta = params['betas']

In [514]:
import trimesh
with open('/home/yuxinyao/datasets/egobody/test/recording_20210910_S06_S05_01/subseq_00000.npz','rb') as f:

    params = dict(np.load(f))
    
bm = smplx.create('/home/yuxinyao/body_models/', model_type='smplx',
                                gender=str(params['gender']), ext='npz',
                                num_pca_comps=12,
                                create_global_orient=True,
                                create_body_pose=True,
                                create_betas=True,
                                create_left_hand_pose=True,
                                create_right_hand_pose=True,
                                create_expression=True,
                                create_jaw_pose=True,
                                create_leye_pose=True,
                                create_reye_pose=True,
                                create_transl=True,
                                batch_size=1
                                )

# bm(params)
# mesh = trimesh.Trimesh(vertices=temp['joints'], faces = )
# e = temp.export(file_type='obj')

In [273]:
# data.keys()

dict_keys(['betas', 'global_orient', 'transl', 'left_hand_pose', 'right_hand_pose', 'jaw_pose', 'leye_pose', 'reye_pose', 'expression', 'pose_embedding', 'body_pose', 'gender'])

In [296]:
with open('/home/yuxinyao/datasets/egobody/test/recording_20210910_S06_S05_01/subseq_00000.npz','rb') as f:
    params = dict(np.load(f))

cmu_canonicalized = dict(np.load('/home/yuxinyao/datasets/AMASS-Canonicalized-locomotion-MPx2/data/CMU/subseq_99999.npz'))
cmu = dict(np.load('/home/yuxinyao/datasets/AMASS/data/CMU/01/01_01_poses.npz'))

In [310]:
params['betas'].shape

(1, 10)

In [515]:
# cmu.keys()
data.keys()

dict_keys(['betas', 'global_orient', 'transl', 'left_hand_pose', 'right_hand_pose', 'jaw_pose', 'leye_pose', 'reye_pose', 'expression', 'pose_embedding', 'body_pose', 'gender'])

In [518]:
# data['body_pose'].shape

In [519]:
data_torch = {}
for k,v in data.items():
    if k=='gender':
        continue
    data_torch[k] = torch.Tensor(v).cuda()



In [321]:
data_torch['betas'].shape

torch.Size([1, 10])

In [323]:
data_torch['global_orient'].shape

torch.Size([1, 3])

In [520]:
bm = bm.cuda()

In [312]:
cmu['poses'].shape

(2751, 156)

In [313]:
# body_param = {}
# body_param['transl'] = torch.FloatTensor(params['trans']).cuda()
# body_param['global_orient'] = torch.FloatTensor([0,0,0]).cuda()
# body_param['betas'] = torch.FloatTensor(betas[:10]).repeat(len_subseq,1).cuda()
# body_param['body_pose'] = torch.FloatTensor(params['poses']).cuda()
# smplxout = bodymodel_batch(return_verts=True, **body_param)



In [521]:
# a = bm(params)
# a = bm(data)
a = bm(**data_torch)


        global_orient.reshape(-1, 1, 3): [torch.Size([1, 1, 3]))],
        body_pose.reshape(-1, self.NUM_BODY_JOINTS, 3):[torch.Size([1, 21, 3])],
        jaw_pose.reshape(-1, 1, 3): [torch.Size([1, 1, 3])]
        leye_pose.reshape(-1, 1, 3): [torch.Size([1, 1, 3])],
        reye_pose.reshape(-1, 1, 3),
        left_hand_pose.reshape(-1, 15, 3),
        right_hand_pose.reshape(-1, 15, 3)]


In [328]:
a.vertices

tensor([[[-0.9214, -0.6199,  2.6119],
         [-0.9199, -0.6197,  2.6159],
         [-0.9196, -0.6212,  2.6162],
         ...,
         [-0.8744, -0.6336,  2.5036],
         [-0.8757, -0.6351,  2.5050],
         [-0.8767, -0.6364,  2.5069]]], device='cuda:0')

In [329]:
bm.faces

array([[    3,     1,     0],
       [    7,     5,     4],
       [   12,    14,    13],
       ...,
       [ 9944, 10097, 10084],
       [ 9940, 10084, 10071],
       [10071, 10058,  9932]], dtype=uint32)

In [333]:
a.vertices.squeeze(0).detach().cpu().numpy()

array([[-0.92138356, -0.61993694,  2.6118927 ],
       [-0.91994363, -0.61973834,  2.6159492 ],
       [-0.91958165, -0.62122405,  2.6162057 ],
       ...,
       [-0.8744125 , -0.6336156 ,  2.5035577 ],
       [-0.87572336, -0.63508177,  2.5050466 ],
       [-0.8766985 , -0.6364423 ,  2.5068622 ]], dtype=float32)

In [343]:
type(a.vertices)

torch.Tensor

In [336]:
# import trimesh 

# t = trimesh.Trimesh(vertices=a.vertices.squeeze(0).detach().cpu().numpy(), faces=bm.faces)
# t.export('/home/yuxinyao/datasets/egobody/test/subseq_00000.obj')

### Visualize

In [542]:
# with open('/home/yuxinyao/datasets/egobody/canicalized-camera-wearer/er/recording_20210910_S06_S05_01/subseq_00040.npz','rb') as f:
# with open('/home/yuxinyao/datasets/egobody/test/recording_20210910_S06_S05_01/subseq_00040.npz','rb') as f:
    # params = dict(np.load(f))
with open("/home/yuxinyao/datasets/egobody/smplx_camera_wearer/recording_20210907_S02_S01_01/body_idx_1/results/frame_01551/000.pkl",'rb') as f:
#     temp = dict(pickle.load(f))
# with open("/home/yuxinyao/datasets/egobody/smplx_interactee/recording_20210907_S02_S01_01/body_idx_0/results/frame_01551/000.pkl",'rb') as f:
    temp = dict(pickle.load(f))


In [543]:
# temp['body_pose'].shape

In [544]:
# with open('/home/yuxinyao/datasets/egobody/test/recording_20210910_S06_S05_01/subseq_00040.npz','rb') as f:
#     temp2 = dict(np.load(f))

In [523]:
# temp['body_pose']

In [535]:
temp.keys()

dict_keys(['betas', 'global_orient', 'transl', 'left_hand_pose', 'right_hand_pose', 'jaw_pose', 'leye_pose', 'reye_pose', 'expression', 'pose_embedding', 'body_pose', 'gender'])

In [539]:
temp_torch = {}
# for k,v in temp.items():
#     if k=='gender':
#         continue
#     temp_torch[k] = torch.Tensor(v).cuda()
for key in temp.keys():
    if key in ['pose_embedding', 'camera_rotation', 'camera_translation', 'gender']:
        continue
    else:
        temp_torch[key] = torch.tensor(temp[key]).cuda()

In [540]:

temp_bm = smplx.create('/home/yuxinyao/body_models/', model_type='smplx',
                            gender=str(temp['gender']), ext='npz',
                            create_body_pose=True,
                            create_betas=True,
                            create_left_hand_pose=True,
                            create_right_hand_pose=True,
                            create_expression=True,
                            create_jaw_pose=True,
                            create_leye_pose=True,
                            create_reye_pose=True,
                            create_transl=True,
                            batch_size=1
                            ).cuda()

In [541]:
b = temp_bm(**temp_torch)

RuntimeError: einsum(): operands do not broadcast with remapped shapes [original->remapped]: [1, 12]->[1, 1, 12] [6, 45]->[1, 45, 6]

In [504]:
# temp.keys()
# temp_params = {}
# temp_params['poses'] = torch.FloatTensor(temp['poses']).cuda()
# temp_params['betas'] = torch.FloatTensor(temp['betas']).repeat(10,1).cuda()
# temp_params['transl'] = torch.FloatTensor(temp['trans']).cuda()
# temp_params['global_orient'] = torch.FloatTensor([0,0,0]).repeat(10,1).cuda()

temp.keys()
temp_params = {}
temp_params['poses'] = torch.FloatTensor(temp['body_pose']).cuda()
temp_params['betas'] = torch.FloatTensor(temp['betas']).cuda()
temp_params['transl'] = torch.FloatTensor(temp['transl']).cuda()
temp_params['global_orient'] = torch.FloatTensor([0,0,0]).cuda()


In [505]:
temp_params['betas'].shape

torch.Size([1, 10])

In [511]:
body_model = smplx.create("/home/yuxinyao/body_models/", model_type='smplx',
                            gender=temp['gender'], ext='npz', 
                            create_global_orient=True, create_body_pose=True, create_betas=True, create_transl=True,
                            create_left_hand_pose=True, create_right_hand_pose=True,
                            create_expression=True, create_jaw_pose=True, create_leye_pose=True, create_reye_pose=True, ).cuda()



In [512]:
temp_params={}

In [530]:
for key in temp.keys():
                if key in ['pose_embedding', 'camera_rotation', 'camera_translation', 'gender']:
                    continue
                else:
                    temp_torch[key] = torch.tensor(temp[key]).cuda()


In [531]:
temp_torch

{'betas': tensor([[-0.8691,  0.3855, -1.1463, -0.6542,  0.0462, -1.1585,  0.9816,  0.7501,
          -0.2685,  0.1033]], device='cuda:0'),
 'global_orient': tensor([[ 0.9271,  0.3762, -3.0154]], device='cuda:0'),
 'transl': tensor([[0.3350, 0.4529, 2.2333]], device='cuda:0'),
 'left_hand_pose': tensor([[-0.0612, -0.2014, -0.0957, -0.4775, -0.2821,  0.0475,  0.1732,  0.1301,
           0.0666,  0.0855, -0.1539, -0.0743]], device='cuda:0'),
 'right_hand_pose': tensor([[-0.1224,  0.0395,  0.0499, -0.2308, -0.1252,  0.0485,  0.0663,  0.0923,
           0.0583,  0.1337, -0.0011,  0.1331]], device='cuda:0'),
 'jaw_pose': tensor([[ 0.0268, -0.0155, -0.0112]], device='cuda:0'),
 'leye_pose': tensor([[-0.2877, -0.3002,  0.2821]], device='cuda:0'),
 'reye_pose': tensor([[-0.2981, -0.0067, -0.1136]], device='cuda:0'),
 'expression': tensor([[ 0.0111, -0.0766,  0.0149,  0.0552,  0.0496,  0.1103,  0.0118, -0.0297,
          -0.0181,  0.0101]], device='cuda:0'),
 'pose_embedding': tensor([[ 1.3907, 

In [527]:
temp_smplxout = temp_bm(**temp_params)

RuntimeError: einsum(): operands do not broadcast with remapped shapes [original->remapped]: [1, 12]->[1, 1, 12] [6, 45]->[1, 45, 6]

In [466]:
# temp_smplxout.vertices.shape

In [467]:
import trimesh
temp_t = trimesh.Trimesh(vertices=temp_smplxout.vertices[0].squeeze(0).detach().cpu().numpy(), faces=temp_bm.faces)

In [468]:
temp_t.export('/home/yuxinyao/datasets/egobody/test/subseq_114.obj')

'# https://github.com/mikedh/trimesh\nv 0.39852232 0.74561286 2.22340512\nv 0.40261099 0.74463832 2.22444892\nv 0.40315291 0.74611032 2.22462058\nv 0.39885545 0.74695587 2.22366953\nv 0.41077718 0.74657351 2.21342397\nv 0.41050905 0.75019908 2.21711469\nv 0.41004086 0.75295466 2.21725869\nv 0.40994516 0.75270796 2.21424580\nv 0.36359757 0.70706046 2.15801382\nv 0.34835860 0.70614278 2.15267611\nv 0.34863466 0.69689322 2.15572572\nv 0.36325386 0.69508654 2.16165876\nv 0.38385999 0.67225683 2.18371916\nv 0.37900731 0.67993087 2.17717838\nv 0.37758335 0.66945612 2.17426777\nv 0.38695601 0.66447961 2.18685222\nv 0.37508228 0.80838805 2.31381679\nv 0.38260946 0.80671358 2.30799627\nv 0.38138500 0.80037248 2.31205177\nv 0.27154383 0.74560940 2.22340035\nv 0.26745516 0.74463463 2.22444415\nv 0.26691297 0.74610651 2.22461557\nv 0.27121028 0.74695230 2.22366476\nv 0.25928771 0.74656904 2.21341872\nv 0.25955564 0.75019467 2.21710944\nv 0.26002473 0.75295019 2.21725321\nv 0.26011953 0.75270355 2.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, glob


'''
In the AMASS dataset, all bodies are located in a bounding box, with
np.min(transall, axis=0) = array([-4.18054399, -3.86190773,  0.00487521])
np.max(transall, axis=0) = array([4.28418131, 4.64069242, 1.91651809])
, which are in terms of meters.
'''


def calc_calibrate_offset(body_mesh_model, betas, transl, pose):
    '''
    The factors to influence this offset is not clear. Maybe it is shape and pose dependent.
    Therefore, we calculate such delta_T for each individual body mesh.
    It takes a batch of body parameters
    input:
        body_params: dict, basically the input to the smplx model
        smplx_model: the model to generate smplx mesh, given body_params
    Output:
        the offset for params transform
    '''
    n_batches = transl.shape[0]
    bodyconfig = {}
    bodyconfig['body_pose'] = torch.FloatTensor(pose[:,3:]).cuda()
    bodyconfig['betas'] = torch.FloatTensor(betas).unsqueeze(0).repeat(n_batches,1).cuda()
    bodyconfig['transl'] = torch.zeros([n_batches,3], dtype=torch.float32).cuda()
    bodyconfig['global_orient'] = torch.zeros([n_batches,3], dtype=torch.float32).cuda()
    smplx_out = body_mesh_model(return_verts=True, **bodyconfig)
    delta_T = smplx_out.joints[:,0,:] # we output all pelvis locations
    delta_T = delta_T.detach().cpu().numpy() #[t, 3]

    return delta_T


def get_new_coordinate(body_mesh_model, betas, transl, pose):
    '''
    this function produces transform from body local coordinate to the world coordinate.
    it takes only a single frame.
    local coodinate:
        - located at the pelvis
        - x axis: from left hip to the right hip
        - z axis: point up (negative gravity direction)
        - y axis: pointing forward, following right-hand rule
    '''
    # print("!!!!!!pose.shape:{}".format(pose.shape))
    bodyconfig = {}
    bodyconfig['transl'] = torch.FloatTensor(transl)
    bodyconfig['global_orient'] = torch.FloatTensor(pose[:,:3])
    bodyconfig['body_pose'] = torch.FloatTensor(pose[:,3:])
    bodyconfig['betas'] = torch.FloatTensor(betas).unsqueeze(0)
    smplxout = body_mesh_model(**bodyconfig)
    joints = smplxout.joints.squeeze().detach().cpu().numpy()
    x_axis = joints[2,:] - joints[1,:]
    x_axis[-1] = 0
    x_axis = x_axis / np.linalg.norm(x_axis)
    z_axis = np.array([0,0,1])
    y_axis = np.cross(z_axis, x_axis)
    y_axis = y_axis/np.linalg.norm(y_axis)
    global_ori_new = np.stack([x_axis, y_axis, z_axis], axis=1)
    transl_new = joints[:1,:] # put the local origin to pelvis

    return global_ori_new, transl_new




def get_body_model(type, gender, batch_size,device='cpu'):
    '''
    type: smpl, smplx smplh and others. Refer to smplx tutorial
    gender: male, female, neutral
    batch_size: an positive integar
    '''
    body_model_path = '/home/yuxinyao/body_models/'
    body_model = smplx.create(body_model_path, model_type=type,
                                    gender=gender, ext='npz',
                                    num_pca_comps=12,
                                    create_global_orient=True,
                                    create_body_pose=True,
                                    create_betas=True,
                                    create_left_hand_pose=True,
                                    create_right_hand_pose=True,
                                    create_expression=True,
                                    create_jaw_pose=True,
                                    create_leye_pose=True,
                                    create_reye_pose=True,
                                    create_transl=True,
                                    batch_size=batch_size
                                    )
    if device == 'cuda':
        return body_model.cuda()
    else:
        return body_model


import sys

if __name__=='__main__':
    N_MPS = 2
    MP_FRAME = 10
    #### set input output dataset paths
    amass_dataset_path = '/home/yuxinyao/datasets/AMASS/data'
    if N_MPS > 1:
        amass_smplx_path = '/home/yuxinyao/datasets/AMASS-Canonicalized-locomotion-MPx{:d}/data'.format(N_MPS)
    else:
        amass_smplx_path = '/home/yuxinyao/datasets/AMASS-Canonicalized-locomotion-MP/data'
    amass_subsets = ['CMU', 'MPI_HDM05', 'BMLmovi', 'KIT', 'Eyes_Japan_Dataset']


    ## set mosh markers
    ## read the corresponding smplx verts indices as markers.
    with open('/home/yuxinyao/body_models/Mosh_related/CMU.json') as f:
            marker_cmu_41 = list(json.load(f)['markersets'][0]['indices'].values())

    with open('/home/yuxinyao/body_models/Mosh_related/SSM2.json') as f:
            marker_ssm_67 = list(json.load(f)['markersets'][0]['indices'].values())

    bm_one_male = get_body_model('smplx','male',1)
    bm_one_female = get_body_model('smplx','female',1)
    

    #### main loop to each subset in AMASS
    for subset in amass_subsets:
        # if not subset in ['HumanEva']:
        #     continue
        seqs = glob.glob(os.path.join(amass_dataset_path, subset, '*/*.npz'))

        outfolder = os.path.join(amass_smplx_path, subset)
        if not os.path.exists(outfolder):
            os.makedirs(outfolder)
        print('-- processing subset {:s}'.format(subset))

        index_subseq = 0 # index subsequences for subsets separately
        #### main loop to process each sequence
        for seq in tqdm(seqs):
            ## read data
            if os.path.basename(seq) == 'shape.npz':
                continue
            data = dict(np.load(seq))
            fps = data['mocap_framerate']
            len_subseq = int(MP_FRAME*N_MPS)
            ## define body model according to gender
            bm_batch_male = get_body_model('smplx','male',len_subseq,device='cuda')
            bm_batch_female = get_body_model('smplx','female',len_subseq,device='cuda')
            bodymodel_batch = bm_batch_male if str(data['gender'].astype(str)) =='male' else bm_batch_female
            bodymodel_one = bm_one_male if str(data['gender'].astype(str)) =='male' else bm_one_female

            ## read data
            transl_all = data['trans']
            pose_all = data['poses']
            betas = data['betas']

            ## skip too short sequences
            n_frames = transl_all.shape[0]
            if n_frames < len_subseq:
                continue
            t = 0
            while t < n_frames:
                ## get subsequence and setup IO
                outfilename = os.path.join(outfolder, 'subseq_{:05d}.npz'.format(index_subseq))
                transl = transl_all[t:t+len_subseq, :]
                pose = pose_all[t:t+len_subseq, :]
                data_out = {}

                ## break if remaining frames are not sufficient
                if transl.shape[0] < len_subseq:
                    break

                ## perform transformation from the world coordinate to the amass coordinate
                ### get transformation from amass space to world space
                transf_rotmat, transf_transl = get_new_coordinate(bodymodel_one, betas[:10], transl[:1,:], pose[:1,:66])
                ### calibrate offset
                delta_T = calc_calibrate_offset(bodymodel_batch, betas[:10], transl, pose[:,:66])
                ### get new global_orient
                global_ori = R.from_rotvec(pose[:,:3]).as_matrix() # to [t,3,3] rotation mat
                global_ori_new = np.einsum('ij,tjk->tik', transf_rotmat.T, global_ori)
                pose[:,:3] = R.from_matrix(global_ori_new).as_rotvec()
                ### get new transl
                transl = np.einsum('ij,tj->ti', transf_rotmat.T, transl+delta_T-transf_transl)-delta_T
                data_out['transf_rotmat'] = transf_rotmat
                data_out['transf_transl'] = transf_transl
                data_out['trans'] = transl
                data_out['poses'] = pose
                data_out['betas'] = betas
                data_out['gender'] = data['gender'].astype(str)
                data_out['mocap_framerate'] = data['mocap_framerate']

                ## under this new amass coordinate, extract the joints/markers' locations
                ## when get generated joints/markers, one can directly transform them back to world coord
                ## note that hand pose is not considered here. In amass, the hand pose is regularized.
                body_param = {}
                body_param['transl'] = torch.FloatTensor(transl).cuda()
                body_param['global_orient'] = torch.FloatTensor(pose[:,:3]).cuda()
                body_param['betas'] = torch.FloatTensor(betas[:10]).unsqueeze(0).repeat(len_subseq,1).cuda()
                body_param['body_pose'] = torch.FloatTensor(pose[:, 3:66]).cuda()
                smplxout = bodymodel_batch(return_verts=True, **body_param)

                # print(type(smplxout))
                # print(type(bm_one_female))
                ### extract joints and markers
                joints = smplxout.joints[:,:22,:].detach().squeeze().cpu().numpy()
                markers_41 = smplxout.vertices[:,marker_cmu_41,:].detach().squeeze().cpu().numpy()
                markers_67 = smplxout.vertices[:,marker_ssm_67,:].detach().squeeze().cpu().numpy()
                data_out['joints'] = joints
                data_out['marker_cmu_41'] = markers_41
                data_out['marker_ssm2_67'] = markers_67

                np.savez(outfilename, **data_out)
                t = t+len_subseq
                index_subseq = index_subseq+1

                print(outfilename)


-- processing subset CMU


  0%|          | 0/2088 [00:01<?, ?it/s]


TypeError: get_new_coordinate() missing 1 required positional argument: 'global_orient'